# LISTA DE FUNCIONES

##Conexión SQLite3 base de datos

Esta función establece una conexión con una base de datos SQLite cuya ruta se pasa como argumento. Si la conexión es exitosa, devuelve tanto el objeto de conexión como el cursor. Si falla, imprime un mensaje de error y retorna None en ambos casos.

Argumentos:
  *   path (str): Ruta del archivo al que se quiere conectar.







In [2]:
def conectar_bd(path):
    """Establece una conexión con la base de datos SQLite."""
    try:
        con = sql.connect(path)
        cursor = con.cursor()
        print(f"Conexión exitosa a la base de datos (copia): {path}")
        return con, cursor
    except sql.Error as e:
        print(f"Error al conectar a la base de datos (copia) {path}: {e}")
        return None, None

## Contar filas de tabla

Esta función cuenta y muestra el número total de filas que existen en una tabla específica de una base de datos SQLite.

Argumentos:
*   cursor (sqlite3.Cursor): Cursor activo asociado a la base de datos.
*   nombre_tabla (str): Nombre de la tabla sobre la cual se quiere contar las filas.



In [3]:
# Contar el número total de filas
def contar_filas_tabla(cursor, nombre_tabla):
    """Cuenta el número total de filas en una tabla."""
    if cursor:
        cursor.execute(f"SELECT COUNT(*) FROM {nombre_tabla};")
        num_filas = cursor.fetchone()[0]
        print(f"\n NÚMERO TOTAL DE FILAS EN '{nombre_tabla}': {num_filas}")
        return num_filas
    return 0

## Conteo de usuarios y peliculas

Calcula la cantidad de usuarios y peliculas que estan presentes en la tabla ratings.

Argumento:
*   cursor (sqlite3.Cursor): Cursor activo asociado a la base de datos.

In [4]:
def conteo_usuarios_peliculas(cursor):
    """Cuenta el número total de usuarios y películas."""
    if cursor:
        cursor.execute("SELECT COUNT(DISTINCT userId), COUNT(DISTINCT movieId) FROM ratings;")
        num_usuarios, num_peliculas = cursor.fetchone()
        print(f"Número total de usuarios únicos: {num_usuarios}")
        print(f"Número total de películas únicas: {num_peliculas}")

## Distribución de calificaciones

Calcula y devuelve la distribución de frecuencias de las calificaciones (rating) registradas en la tabla ratings, organizada en un DataFrame de Pandas.

Argumentos:
*   cursor (sqlite3.Cursor): Cursor activo asociado a la base de datos.



In [5]:
def distribucion_calificaciones(cursor):
    """Devuelve la distribución de calificaciones en un DataFrame."""
    if cursor:
        cursor.execute("SELECT rating, COUNT(*) AS frecuencia FROM ratings GROUP BY rating ORDER BY rating;")
        distribucion = cursor.fetchall()
        df_distribucion = pd.DataFrame(distribucion, columns=['rating', 'frecuencia'])
        return df_distribucion
    return pd.DataFrame(columns=['rating', 'frecuencia'])

## Explorar esquema de tabla

Esta función imprime el esquema de una tabla específica dentro de una base de datos SQLite. El esquema incluye el nombre de cada columna, su tipo de dato y si permite valores nulos.

Argumentos:
*   cursor (sqlite3.Cursor): Cursor activo asociado a una conexión SQLite
*   nombre_tabla (str): Nombre de la tabla de la cual se quiere inspeccionar el esquema.

In [6]:
# Ver el esquema de la tabla
def explorar_esquema_tabla(cursor, nombre_tabla):
    """Muestra el esquema de una tabla específica."""
    if cursor:
        print(f"\n ESQUEMA DE LA TABLA '{nombre_tabla}'")
        cursor.execute(f"PRAGMA table_info('{nombre_tabla}');")
        columnas = cursor.fetchall()
        for columna in columnas:
            print(f"\n  - Nombre Columna: {columna[1]}")
            print(f"    Tipo de Dato: {columna[2]}")
            print(f"    ¿Permite Valores Nulos? (0 es sí - 1 es no): {columna[3]}")

## Listar tablas

Esta función consulta y muestra los nombres de todas las tablas disponibles en una base de datos SQLite, usando el cursor proporcionado. También devuelve una lista con esos nombres.

Argumentos:
*   cursor (sqlite3.Cursor): Cursor activo asociado a una conexión SQLite.

In [7]:
def listar_tablas(cursor):
    """Lista las tablas disponibles en la base de datos."""
    if cursor:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tablas = cursor.fetchall()
        print("\nTablas encontradas en la base de datos:")
        for tabla in tablas:
            print(f"* {tabla[0]}")
        return [t[0] for t in tablas]
    return []

## Moda

Calcula la moda de una lista de números o elementos. Retorna None si la lista está vacía. Si hay múltiples modas, retorna la primera encontrada por Counter.

Argumento:
* lista (list): Una lista de valores.

In [8]:
def calcular_moda(lista):
    if not lista:
        return None
    contador = Counter(lista)
    moda = contador.most_common(1)[0][0]
    return moda

## Primeras filas de una tabla

Esta función muestra las primeras n filas de una tabla en una base de datos SQLite utilizando una consulta SQL.

Argumentos:
*   cursor (sqlite3.Cursor): Cursor activo para ejecutar la consulta SQL.
*   nombre_tabla (str): Nombre de la tabla desde la cual se quieren visualizar las filas.
*   num_filas (int, opcional): Número de filas a mostrar. Por defecto es 5.



In [9]:
# Mostrar las primeras 5 filas
def mostrar_primeras_filas_sql(cursor, nombre_tabla, num_filas=5):
    """Muestra las primeras N filas de una tabla usando SQL."""
    if cursor:
        cursor.execute(f"SELECT * FROM {nombre_tabla} LIMIT {num_filas};")
        primeras_filas = cursor.fetchall()
        if primeras_filas:
            print(f"\n PRIMERAS {num_filas} FILAS DE LA TABLA '{nombre_tabla}'")
            for fila in primeras_filas:
                print(fila)
        else:
            print(f"\nLA TABLA '{nombre_tabla}' ESTÁ VACÍA.")

## Separar géneros

Función que toma el DataFrame de películas con una columna de géneros separada por '|' y la expande en columnas booleanas para cada género único.

Argumentos:
*   df (pd.DataFrame): El DataFrame de películas con la columna 'genres'.

Retorna:
*   pd.DataFrame: Un nuevo DataFrame con las columnas de género añadidas (como enteros 0 o 1) y la columna 'genres' eliminada.



In [10]:
def separar_generos(df):
    """Separa la columna de géneros en columnas booleanas."""
    genres_series = df['genres'].str.split('|')
    te = TransactionEncoder()
    genres_array = te.fit(genres_series).transform(genres_series)
    df_genres = pd.DataFrame(genres_array, columns=te.columns_)
    df_processed = pd.concat([df, df_genres], axis=1).drop(columns='genres')
    df_processed = df_processed.astype({col: 'int' for col in df_genres.columns if df_genres[col].dtype == 'bool'})
    return df_processed